In [1]:
# Imporing libraries
import os
import pandas as pd
import numpy as np
import statistics as stat
import itertools
from functools import reduce

## Read the dataset, display the first few rows in the dataset.

In [2]:
# Read the ratings dataset
baseLocation = "D:/TUNI/Courses/Period-2/DATA.ML.360 [Recommender]/Assignment 1/ml-latest-small/"
ratings_data = pd.read_csv(baseLocation+"ratings.csv")

# Displaying top 5 ratings data
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Remove 'timestamp' column from ratings dataset because it's not needed
ratings_data.drop('timestamp', inplace=True, axis=1)

# Displaying top 5 ratings data
ratings_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
# Count of ratings (rows)
print(len(ratings_data))

# Count of users
print(len(pd.unique(ratings_data['userId'])))

100836
610


In [5]:
# Read the movie dataset
movies_data = pd.read_csv(baseLocation+'movies.csv')

# Displaying top 5 movies data
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Remove the "genres" column from the movies dataset because it is unnecessary
movies_data.drop('genres', inplace=True, axis=1)

# Displaying top 5 ratings data
movies_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


## Implement the user-based collaborative filtering approach, using the Pearson correlation function for computing similarities between users

In [7]:
# We have chosen 3 users (35,135,235)
group = [35,135,235]

In [8]:
# Getting all the movies of a selected group has seen and rated
inputMovies=[]

for user in group:
    uId = user
    Movies = ratings_data[ratings_data['userId']== uId]
    inputMovies.append(Movies)

# Top five movies of the users of the group has seen or rated
for i in range(len(inputMovies)):
    print(len(inputMovies[i]))
    print(inputMovies[i].head())

23
      userId  movieId  rating
5273      35       11     4.0
5274      35       21     5.0
5275      35       39     3.0
5276      35       50     5.0
5277      35       60     5.0
279
       userId  movieId  rating
20336     135        1     4.0
20337     135        2     3.0
20338     135       21     3.0
20339     135       32     5.0
20340     135       47     5.0
65
       userId  movieId  rating
35112     235       10     2.0
35113     235       11     4.0
35114     235       21     4.0
35115     235       25     5.0
35116     235       32     4.0


In [9]:
# Similar users like our selected users of the group
similarUsers = []
for i in range(len(inputMovies)):
    tempSimilarUsers = ratings_data[ratings_data['movieId'].isin(inputMovies[i]['movieId'].tolist())]
    similarUsers.append(tempSimilarUsers)

# Top 5 elements of the similar users
for i in range(len(similarUsers)):
    print(similarUsers[i].head())

    userId  movieId  rating
4        1       50     5.0
13       1      231     5.0
14       1      235     4.0
17       1      316     3.0
32       1      590     4.0
    userId  movieId  rating
0        1        1     4.0
3        1       47     5.0
7        1      110     4.0
9        1      157     5.0
12       1      223     3.0
    userId  movieId  rating
7        1      110     4.0
12       1      223     3.0
14       1      235     4.0
16       1      296     3.0
19       1      349     4.0


In [10]:
# Sub dataframes based on users of the group
similarUsersGroup = []
for i in range(len(similarUsers)):
    tempsimilarUsersGroup = similarUsers[i].groupby(['userId'])
    similarUsersGroup.append(tempsimilarUsersGroup)

In [11]:
# Sort dataframes
for i in range(len(similarUsersGroup)): 
    similarUsersGroup[i] = sorted(similarUsersGroup[i], key= lambda x:len(x[1]), reverse = True)

In [12]:
# 2nd top user for each user of the group (1st one is the same as our users of the group)
for i in range(len(similarUsersGroup)):
    print(similarUsersGroup[i][1])

(117,        userId  movieId  rating
18355     117       11     4.0
18358     117       21     4.0
18364     117       39     3.0
18369     117       50     4.0
18370     117       60     3.0
18371     117       62     4.0
18376     117      150     4.0
18385     117      185     3.0
18393     117      231     2.0
18394     117      235     3.0
18395     117      236     3.0
18396     117      237     3.0
18398     117      252     4.0
18401     117      261     3.0
18403     117      266     4.0
18413     117      300     4.0
18415     117      316     3.0
18420     117      339     3.0
18466     117      590     4.0
18467     117      592     3.0
18470     117      595     4.0)
(414,        userId  movieId  rating
62294     414        1     4.0
62295     414        2     3.0
62307     414       21     4.0
62314     414       32     5.0
62322     414       47     4.0
...       ...      ...     ...
63774     414     4776     4.0
63779     414     4812     3.0
63797     414     4881    

In [13]:
# Number of similar users
for i in range(len(similarUsersGroup)):
    print(len(similarUsersGroup[i]))

432
604
565


In [14]:
# Number of movies of the first most similar user
for i in range(len(similarUsersGroup)):
    print(len(similarUsersGroup[i][1][1]))

21
238
57


## Calculating The Similarity (Pearson Correlation)

In [15]:
# Number of similar users (432, 604, 565) but not all of them contribute to the recommendation.
# Here, limiting the number to 100 Similar Users
for i in range(len(similarUsersGroup)):
    similarUsersGroup[i] = similarUsersGroup[i][1:101] # Removing the first user (userId = 35,135,235), as it is our selected user
    print(len(similarUsersGroup[i]))

100
100
100


In [16]:
# List of correlations of the users of the group
pearsonCorrelation = []

for i in range(len(similarUsersGroup)):
    # Store the Pearson Correlation in a dictionary, 
    pearsonCorrelationDict = {}
    
    # Get movies list of a user of the group and sort them by movieId
    temp_inputMovies = inputMovies[i].sort_values(by='movieId')
    
    # For every similar user group to an individual user of the group in our subset
    for name, group in similarUsersGroup[i]:
    
        # Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='movieId')
    
        # Get the review scores of the Selected user for the movies that they both have in common
        temp_df = temp_inputMovies[temp_inputMovies['movieId'].isin(group['movieId'].tolist())]
    
        # And then store them in a temporary buffer variable in a list format to facilitate future calculations
        selectedUserTempRatingList = temp_df['rating'].tolist()
    
        # Let's also put the current user group reviews in a list format
        similarUserTempRatingList = group['rating'].tolist()
    
        # Calculating pearson similarity
        simXX = 0
        meanSelectedUserTempRating = stat.mean(selectedUserTempRatingList)
        for i in selectedUserTempRatingList:
            simXX = simXX + pow((i - meanSelectedUserTempRating),2)
    
        simYY = 0
        meanSimilarUserTempRaning = stat.mean(similarUserTempRatingList)
        for j in similarUserTempRatingList:
            simYY = simYY + pow((j - meanSimilarUserTempRaning),2)
        
        simXY = 0
        for i, j in zip(selectedUserTempRatingList, similarUserTempRatingList):
            simXY = simXY+ ((i - meanSelectedUserTempRating ) * (j-meanSimilarUserTempRaning))
    
        # If the denominator is different than zero, then divide, else, 0 correlation.
        if simXX != 0 and simYY != 0:
            pearsonCorrelationDict[name] = simXY/np.sqrt(simXX*simYY)
        else:
            pearsonCorrelationDict[name] = 0                    
    
    pearsonCorrelation.append(pearsonCorrelationDict)

In [17]:
# Correlation Values
for i in range(len(pearsonCorrelation)):
    print(pearsonCorrelation[i].items())
    print(" ")

dict_items([(117, 0.1899901489143371), (414, -0.038335206136624694), (474, 0.17952413671328593), (6, -0.16836470741027082), (411, 0.4828078792603351), (38, 0.4561749161023165), (446, 0.36531753932878963), (599, -0.06146362971528592), (58, -0.19122793204774682), (458, 0.05454545454545454), (566, 0.23624976928744731), (40, 0.45431304586442517), (68, -0.5216955557265005), (84, 0.06736259938424739), (284, -0.4783446031074604), (429, 0.5264520511569365), (470, -0.07850686719788592), (480, -0.053946265454997186), (584, 0.5725820947661028), (602, 0.027532438555856614), (91, 0.24116846298044733), (174, -0.10663040478661669), (307, -0.08593000782454135), (314, 0.11932326792871528), (489, -0.13445852909056286), (8, 0.5231388366621641), (43, 0.3175426480542942), (45, -0.11012204898040079), (109, 0.35120668762232365), (136, 0.32313723196612754), (372, -0.2707638067882481), (373, 0.01747259921139871), (455, 0.012515654358043936), (600, -0.2335189460269705), (46, 0.2124307787987451), (121, 0.3492151

In [18]:
similarityScoreofSimilarUsers_data =[]
for i in range(len(pearsonCorrelation)):
    tempSimilarityScoreofSimilarUsers_data = pd.DataFrame.from_dict(pearsonCorrelation[i], orient='index')
    similarityScoreofSimilarUsers_data.append(tempSimilarityScoreofSimilarUsers_data)
    
for i in range(len(similarityScoreofSimilarUsers_data)):
    print(similarityScoreofSimilarUsers_data[i].head())

            0
117  0.189990
414 -0.038335
474  0.179524
6   -0.168365
411  0.482808
            0
414  0.366691
599  0.345551
474  0.371757
448  0.338578
68   0.126779
            0
414  0.449784
6    0.317486
117  0.463710
599  0.380023
474  0.391199


In [19]:
# Similarity score of other similar users to our selected user (35,135,235)
for i in range(len(similarityScoreofSimilarUsers_data)):
    similarityScoreofSimilarUsers_data[i].columns = ['similarityScore']
    similarityScoreofSimilarUsers_data[i]['userId'] = similarityScoreofSimilarUsers_data[i].index
    similarityScoreofSimilarUsers_data[i].index = range(len(similarityScoreofSimilarUsers_data[i]))
    
for i in range(len(similarityScoreofSimilarUsers_data)):
    print(similarityScoreofSimilarUsers_data[i].head())  

   similarityScore  userId
0         0.189990     117
1        -0.038335     414
2         0.179524     474
3        -0.168365       6
4         0.482808     411
   similarityScore  userId
0         0.366691     414
1         0.345551     599
2         0.371757     474
3         0.338578     448
4         0.126779      68
   similarityScore  userId
0         0.449784     414
1         0.317486       6
2         0.463710     117
3         0.380023     599
4         0.391199     474


In [20]:
# Top Similar users
topSimilarUsers = []
for i in range(len(similarityScoreofSimilarUsers_data)):
    tempTopSimilarUsers=similarityScoreofSimilarUsers_data[i].sort_values(by='similarityScore', ascending=False)
    topSimilarUsers.append(tempTopSimilarUsers)
    
for i in range(len(topSimilarUsers)):
    print(topSimilarUsers[i].head(5))

    similarityScore  userId
97         0.784465     305
92         0.700649     179
77         0.682500     436
89         0.674200     107
85         0.672194     606
    similarityScore  userId
69         0.528075     428
15         0.526236     387
65         0.518887     434
84         0.511239     596
95         0.499403     325
    similarityScore  userId
50         0.616588      40
15         0.578526      84
21         0.577974     590
60         0.567224     570
81         0.544923     179


## Calculating Recommendation Score

In [21]:
# Merging the top similar users similarity score with their ratings of movies
topSimilarUsersRating =[]
for i in range(len(topSimilarUsers)):
    tempTopSimilarUsersRating = topSimilarUsers[i].merge(ratings_data, left_on='userId', right_on='userId', how='inner')
    topSimilarUsersRating.append(tempTopSimilarUsersRating)
    
for i in range(len(topSimilarUsersRating)):  
    print(topSimilarUsersRating[i].head())

   similarityScore  userId  movieId  rating
0         0.784465     305        2     3.5
1         0.784465     305        6     3.5
2         0.784465     305       16     4.5
3         0.784465     305       25     3.0
4         0.784465     305       32     5.0
   similarityScore  userId  movieId  rating
0         0.528075     428        9     2.0
1         0.528075     428       19     1.0
2         0.528075     428       22     2.5
3         0.528075     428       44     2.0
4         0.528075     428       69     3.5
   similarityScore  userId  movieId  rating
0         0.616588      40        1     5.0
1         0.616588      40       17     5.0
2         0.616588      40       19     2.0
3         0.616588      40       21     4.0
4         0.616588      40       26     4.0


In [22]:
# Calculating the mean ratings of all the similar users (suppose, b is a similar user to a group-user)
meanRb = []

for i in range(len(topSimilarUsersRating)):
    tempMeanRb = topSimilarUsersRating[i].groupby('userId').mean()[['rating']]
    tempMeanRb.columns = ['avgRating']
    tempMeanRb['userId'] = tempMeanRb.index
    tempMeanRb.index = range(len(tempMeanRb))
    
    meanRb.append(tempMeanRb)

for i in range(len(meanRb)):
    print(meanRb[i].head())

   avgRating  userId
0   3.636364       5
1   3.493631       6
2   3.574468       8
3   3.754902      32
4   3.788462      33
   avgRating  userId
0   4.366379       1
1   3.555556       4
2   3.732072      18
3   2.607397      19
4   3.260722      21
   avgRating  userId
0   3.636364       5
1   3.493631       6
2   3.574468       8
3   3.732072      18
4   2.607397      19


In [23]:
# Merging the top similar users average rating score 
for i in range(len(topSimilarUsersRating)):
    topSimilarUsersRating[i] = topSimilarUsersRating[i].merge(meanRb[i], left_on='userId', right_on='userId', how='inner')
    
for i in range(len(topSimilarUsersRating)):
    print(topSimilarUsersRating[i].head())

   similarityScore  userId  movieId  rating  avgRating
0         0.784465     305        2     3.5   3.924668
1         0.784465     305        6     3.5   3.924668
2         0.784465     305       16     4.5   3.924668
3         0.784465     305       25     3.0   3.924668
4         0.784465     305       32     5.0   3.924668
   similarityScore  userId  movieId  rating  avgRating
0         0.528075     428        9     2.0       2.64
1         0.528075     428       19     1.0       2.64
2         0.528075     428       22     2.5       2.64
3         0.528075     428       44     2.0       2.64
4         0.528075     428       69     3.5       2.64
   similarityScore  userId  movieId  rating  avgRating
0         0.616588      40        1     5.0    3.76699
1         0.616588      40       17     5.0    3.76699
2         0.616588      40       19     2.0    3.76699
3         0.616588      40       21     4.0    3.76699
4         0.616588      40       26     4.0    3.76699


In [24]:
# Multiplies the similarity by the user's ratings
for i in range(len(topSimilarUsersRating)):
    topSimilarUsersRating[i]['weightedRatingScore'] = topSimilarUsersRating[i]['similarityScore']*(topSimilarUsersRating[i]['rating']-topSimilarUsersRating[i]['avgRating'])

for i in range(len(topSimilarUsersRating)):
    print(topSimilarUsersRating[i].head())

   similarityScore  userId  movieId  rating  avgRating  weightedRatingScore
0         0.784465     305        2     3.5   3.924668            -0.333137
1         0.784465     305        6     3.5   3.924668            -0.333137
2         0.784465     305       16     4.5   3.924668             0.451328
3         0.784465     305       25     3.0   3.924668            -0.725369
4         0.784465     305       32     5.0   3.924668             0.843560
   similarityScore  userId  movieId  rating  avgRating  weightedRatingScore
0         0.528075     428        9     2.0       2.64            -0.337968
1         0.528075     428       19     1.0       2.64            -0.866043
2         0.528075     428       22     2.5       2.64            -0.073931
3         0.528075     428       44     2.0       2.64            -0.337968
4         0.528075     428       69     3.5       2.64             0.454145
   similarityScore  userId  movieId  rating  avgRating  weightedRatingScore
0         0.

In [25]:
# Applies a sum to the topUsers after grouping it up by userId
topSimilarUsersRatingForeachMovie = []

for i in range(len(topSimilarUsersRating)):
    tempTopSimilarUsersRating = topSimilarUsersRating[i].groupby('movieId').sum()[['weightedRatingScore']]
    tempTopSimilarUsersRating.columns = ['sum_weightedRatingScore']
    tempTopSimilarUsersRating['movieId'] = tempTopSimilarUsersRating.index
    
    topSimilarUsersRatingForeachMovie.append(tempTopSimilarUsersRating)
    
for i in range(len(topSimilarUsersRatingForeachMovie)):  
    print(topSimilarUsersRatingForeachMovie[i].head()) 

         sum_weightedRatingScore  movieId
movieId                                  
1                       1.196120        1
2                       0.078638        2
3                       1.064800        3
4                      -0.645703        4
5                      -0.245415        5
         sum_weightedRatingScore  movieId
movieId                                  
1                       8.591931        1
2                      -2.844434        2
3                      -1.019347        3
4                      -0.444491        4
5                      -3.283965        5
         sum_weightedRatingScore  movieId
movieId                                  
1                      12.140254        1
2                       0.976881        2
3                      -0.400920        3
4                      -1.129472        4
5                      -5.267823        5


In [26]:
# Creates an empty list of dataframes to store the final recommendation score
recommendation_data = []

for i in range(len(topSimilarUsersRatingForeachMovie)):
    # Create an empty dataframe
    temp_recommendation_data = pd.DataFrame()
    
    # Now we take the weighted average
    meanRa = inputMovies[i]['rating'].mean()
    temp_recommendation_data['weighted average recommendation score'] = meanRa+(topSimilarUsersRatingForeachMovie[i]['sum_weightedRatingScore']/topSimilarUsers[i]['similarityScore'].sum())
    temp_recommendation_data['movieId'] = temp_recommendation_data.index
    
    # Removing the name of the index for future ambiguity
    temp_recommendation_data.index.name = None
    
    # Swaping Coumns
    titles = list(temp_recommendation_data)
    titles[0], titles[1] = titles[1], titles[0]
    temp_recommendation_data = temp_recommendation_data[titles]
    
    recommendation_data.append(temp_recommendation_data)

for i in range(len(recommendation_data)):    
    print(recommendation_data[i].head())

   movieId  weighted average recommendation score
1        1                               4.159203
2        2                               4.091706
3        3                               4.151271
4        4                               4.047956
5        5                               4.072133
   movieId  weighted average recommendation score
1        1                               3.938944
2        2                               3.528819
3        3                               3.594269
4        4                               3.614884
5        5                               3.513057
   movieId  weighted average recommendation score
1        1                               4.064548
2        2                               3.679820
3        3                               3.632337
4        4                               3.607228
5        5                               3.464607


## Recommended Movies 

In [27]:
# Sort the recommendation scores and recommend movies for a user of the group
for i in range(len(recommendation_data)):
    recommendation_data[i] = recommendation_data[i].sort_values(by='weighted average recommendation score', ascending=False)
    recommendation_data[i] = recommendation_data[i].reset_index(drop=True)
    recommendation_data[i]['rank'] = recommendation_data[i].index
    
    print(recommendation_data[i].head())

   movieId  weighted average recommendation score  rank
0      318                               4.954681     0
1       50                               4.791123     1
2      296                               4.648732     2
3      527                               4.623802     3
4      150                               4.585603     4
   movieId  weighted average recommendation score  rank
0      296                               4.522742     0
1     1196                               4.476703     1
2      260                               4.444333     2
3      858                               4.409846     3
4     2571                               4.390945     4
   movieId  weighted average recommendation score  rank
0      318                               4.599339     0
1      356                               4.427028     1
2      527                               4.333266     2
3      593                               4.293626     3
4      110                               4.26462

## Top 20 Recommended Movies (Individual Users of the group)

In [28]:
# Top 20 Recommended Movies
for i in range(len(recommendation_data)):
    print("User :", i)
    print(movies_data.loc[movies_data['movieId'].isin(recommendation_data[i].head(20)['movieId'].tolist())])

User : 0
      movieId                             title
15         16                     Casino (1995)
32         34                       Babe (1995)
46         50        Usual Suspects, The (1995)
55         62         Mr. Holland's Opus (1995)
97        110                 Braveheart (1995)
123       150                  Apollo 13 (1995)
190       223                     Clerks (1994)
257       296               Pulp Fiction (1994)
277       318  Shawshank Redemption, The (1994)
314       356               Forrest Gump (1994)
398       457              Fugitive, The (1993)
434       497     Much Ado About Nothing (1993)
461       527           Schindler's List (1993)
508       590         Dances with Wolves (1990)
510       593  Silence of the Lambs, The (1991)
706       924      2001: A Space Odyssey (1968)
907      1206        Clockwork Orange, A (1971)
1662     2232                       Cube (1997)
1939     2571                Matrix, The (1999)
2460     3273                  

In [29]:
# Combining all users ratings by movieId
temp_group_recommendation_data = reduce(lambda left,right: pd.merge(left,right,on='movieId'), recommendation_data)
temp_group_recommendation_data.head(20)

,movieId,weighted average recommendation score_x,rank_x,weighted average recommendation score_y,rank_y,weighted average recommendation score,rank
0,318,4.954681,0,4.295213,6,4.599339,0
1,50,4.791123,1,4.259269,9,4.083112,10
2,296,4.648732,2,4.522742,0,4.230116,5
3,527,4.623802,3,4.201946,16,4.333266,2
4,150,4.585603,4,3.725229,360,3.941335,26
5,590,4.543257,5,3.750337,284,3.900709,36
6,593,4.492552,6,4.219574,15,4.293626,3
7,223,4.463540,7,3.772183,239,3.841998,67
8,924,4.454826,8,4.006904,56,3.858866,51
9,457,4.448615,9,3.864439,113,4.170016,6


## Aggregation : Average Method

In [30]:
# Calculate average score and sort them
final_avg_group_recommendation_data = temp_group_recommendation_data.copy(deep = False)
final_avg_group_recommendation_data['Average group recommendation score']= final_avg_group_recommendation_data[['weighted average recommendation score_x', 'weighted average recommendation score_y', 'weighted average recommendation score']].mean(axis=1)
final_avg_group_recommendation_data = final_avg_group_recommendation_data.sort_values(by=['Average group recommendation score'], ascending=False)

final_avg_group_recommendation_data = final_avg_group_recommendation_data.reset_index(drop=True)
final_avg_group_recommendation_data['final_rank'] = final_avg_group_recommendation_data.index

final_avg_group_recommendation_data.head(20)

,movieId,weighted average recommendation score_x,rank_x,weighted average recommendation score_y,rank_y,weighted average recommendation score,rank,Average group recommendation score,final_rank
0,318,4.954681,0,4.295213,6,4.599339,0,4.616411,0
1,296,4.648732,2,4.522742,0,4.230116,5,4.467196,1
2,527,4.623802,3,4.201946,16,4.333266,2,4.386338,2
3,50,4.791123,1,4.259269,9,4.083112,10,4.377835,3
4,356,4.435531,10,4.163270,17,4.427028,1,4.341943,4
5,593,4.492552,6,4.219574,15,4.293626,3,4.335251,5
6,260,4.247576,45,4.444333,2,4.093168,9,4.261692,6
7,858,4.258048,36,4.409846,3,4.059174,12,4.242356,7
8,110,4.417545,11,4.044053,41,4.264628,4,4.242075,8
9,2571,4.311385,18,4.390945,4,4.022143,14,4.241491,9


## Top-20 Recommendations (Movies) for the group : Average Method

In [31]:
# Top 20 Recommended Movies
movies_data.loc[movies_data['movieId'].isin(final_avg_group_recommendation_data.head(20)['movieId'].tolist())]

,movieId,title
46,50,"Usual Suspects, The (1995)"
97,110,Braveheart (1995)
224,260,Star Wars: Episode IV - A New Hope (1977)
257,296,Pulp Fiction (1994)
277,318,"Shawshank Redemption, The (1994)"
314,356,Forrest Gump (1994)
322,364,"Lion King, The (1994)"
398,457,"Fugitive, The (1993)"
461,527,Schindler's List (1993)
507,589,Terminator 2: Judgment Day (1991)


## Aggregation : Least Misery Method

In [32]:
# Calculate the min Ratings of the group (Least Misery Approach)
final_min_group_recommendation_data = temp_group_recommendation_data.copy(deep = False)
final_min_group_recommendation_data['Min group recommendation score']= final_min_group_recommendation_data[['weighted average recommendation score_x', 'weighted average recommendation score_y', 'weighted average recommendation score']].min(axis=1)
final_min_group_recommendation_data = final_min_group_recommendation_data.sort_values(by=['Min group recommendation score'], ascending=False)

final_min_group_recommendation_data = final_min_group_recommendation_data.reset_index(drop=True)
final_min_group_recommendation_data['final_rank'] = final_min_group_recommendation_data.index

final_min_group_recommendation_data.head(20)

,movieId,weighted average recommendation score_x,rank_x,weighted average recommendation score_y,rank_y,weighted average recommendation score,rank,Min group recommendation score,final_rank
0,318,4.954681,0,4.295213,6,4.599339,0,4.295213,0
1,296,4.648732,2,4.522742,0,4.230116,5,4.230116,1
2,593,4.492552,6,4.219574,15,4.293626,3,4.219574,2
3,527,4.623802,3,4.201946,16,4.333266,2,4.201946,3
4,356,4.435531,10,4.163270,17,4.427028,1,4.163270,4
5,260,4.247576,45,4.444333,2,4.093168,9,4.093168,5
6,589,4.132462,576,4.089331,29,4.150897,7,4.089331,6
7,50,4.791123,1,4.259269,9,4.083112,10,4.083112,7
8,858,4.258048,36,4.409846,3,4.059174,12,4.059174,8
9,110,4.417545,11,4.044053,41,4.264628,4,4.044053,9


## Top-20 Recommendations (Movies) for the group : Least Misery Method

In [33]:
# Top 20 Recommended Movies (least Misery apporach)
movies_data.loc[movies_data['movieId'].isin(final_min_group_recommendation_data.head(20)['movieId'].tolist())]

,movieId,title
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
43,47,Seven (a.k.a. Se7en) (1995)
46,50,"Usual Suspects, The (1995)"
97,110,Braveheart (1995)
224,260,Star Wars: Episode IV - A New Hope (1977)
257,296,Pulp Fiction (1994)
277,318,"Shawshank Redemption, The (1994)"
314,356,Forrest Gump (1994)
461,527,Schindler's List (1993)
507,589,Terminator 2: Judgment Day (1991)


 ## Part B: Disagreements

### Compute Kendall tau Distance for both the aggregation method (Average and Least Misery)

In [34]:
# Defining Kendall Tau Distance Function:
def kendallTau(A, B):
    pairs = itertools.combinations(range(0, len(A)), 2)

    distance = 0

    for x, y in pairs:
        a = A[x] - A[y]
        b = B[x] - B[y]

        # if discordant (different signs)
        if (a * b < 0):
            distance += 1

    return distance

In [35]:
# Creating the ranking list of the users and the average aggregation method
AvgDisagg = final_avg_group_recommendation_data.copy(deep= False) 
AvgDisagg = AvgDisagg.sort_values(by=["movieId"])

r1 = AvgDisagg['rank_x'].tolist()
r2 = AvgDisagg['rank_y'].tolist()
r3 = AvgDisagg['rank'].tolist()
ra = AvgDisagg['final_rank'].tolist()

In [36]:
# Creating the ranking list of the users and the least miseary aggregation method
LmDisagg = final_min_group_recommendation_data.copy(deep= False) 
LmDisagg = LmDisagg.sort_values(by=["movieId"])

r1 = LmDisagg['rank_x'].tolist()
r2 = LmDisagg['rank_y'].tolist()
r3 = LmDisagg['rank'].tolist()
rlm = LmDisagg['final_rank'].tolist()

In [37]:
TauAvg =kendallTau(r1,ra)+kendallTau(r2,ra)+kendallTau(r3,ra)
TauLm = kendallTau(r1,rlm)+kendallTau(r2,rlm)+kendallTau(r3,rlm)

print(TauAvg)
print(TauLm)

17563761
16718478


In [38]:
# Top 20 Recommended Movies
movies_data.loc[movies_data['movieId'].isin(final_avg_group_recommendation_data.head(20)['movieId'].tolist())]

,movieId,title
46,50,"Usual Suspects, The (1995)"
97,110,Braveheart (1995)
224,260,Star Wars: Episode IV - A New Hope (1977)
257,296,Pulp Fiction (1994)
277,318,"Shawshank Redemption, The (1994)"
314,356,Forrest Gump (1994)
322,364,"Lion King, The (1994)"
398,457,"Fugitive, The (1993)"
461,527,Schindler's List (1993)
507,589,Terminator 2: Judgment Day (1991)
